In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD, LinearSolve
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

integ_set = (alg = Vern9(), adaptive = true, abstol = 1e-15, reltol = 1e-15);

In [4]:
index_hom = 3
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [5]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)
ϵ_sep = 1e-6
ϵ_box = 1e-7
Ju0 = jacob_TM_(fp[1], p, 0);

In [6]:
eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors
v1 = real(eigen_vectors[:, 1])

dot_on_sep_stable = fp[1] + v1*ϵ_sep

3-element SVector{3, Float64} with indices SOneTo(3):
 8.345789408165038
 0.7384945878731853
 0.43829843650314865

In [7]:
t = 1000.0; tt = 1000.0; tstep = 0.001; tspan = [0.0, t]; tspanback = [0.0, -t]

2-element Vector{Float64}:
     0.0
 -1000.0

In [8]:
# forward
prob_for = ODEProblem(TM, dot_on_sep_stable, tspan, p)
sol_for = solve(prob_for, Vern9(), adaptive = true, abstol = 1e-15, reltol = 1e-15);

In [9]:
prob_back = ODEProblem(TM, dot_on_sep_stable, tspanback, p) # backward time
sol_back = solve(prob_back, Rodas5(), adaptive = true, abstol = 1e-14, reltol = 1e-14)

prob_back_con = ODEProblem(TM, sol_for[5040], tspanback, p) # backward time, u0 from forward time
sol_back_con = solve(prob_back_con, Rodas5(), adaptive = true, abstol = 1e-14, reltol = 1e-14)

┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


retcode: Unstable
Interpolation: specialized 4rd order "free" stiffness-aware interpolation
t: 137074-element Vector{Float64}:
   0.0
  -0.00025955722823212254
  -0.0003973250796084803
  -0.0006074362851383878
  -0.0008175474906682951
  -0.0010384361627116933
  -0.00126310639044116
  -0.0014937487976952317
  -0.0017281139704840183
  -0.001966539386152735
   ⋮
 -15.404965607974
 -15.405037745786123
 -15.405109883598245
 -15.405182021410367
 -15.405254159222489
 -15.40532629703461
 -15.405398434846733
 -15.405470572658855
 -15.405542710470977
u: 137074-element Vector{SVector{3, Float64}}:
 [16.153597394614057, 0.7339943621431889, 0.5850778119779825]
 [16.095868210465976, 0.7348840111696783, 0.5850909073226342]
 [16.065157337659933, 0.7353557012226687, 0.5850977210656534]
 [16.018229476642258, 0.7360743716243219, 0.585107929648878]
 [15.971194915714223, 0.7367921664317653, 0.58511791684691]
 [15.921635781817713, 0.7375458122983063, 0.5851281774334821]
 [15.87111377045891, 0.73831131792121

In [22]:
tsbcon, tfbcon = 1, 7800 # time interval backward u0 from forward time
tsb, tfb = 1, 3500 # time interval backward

tsf, tff = 1, 5500 # time interval forward
idx, idy, idz = 2, 3, 1
dis  = 60;

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 50, 50, 1)

lines!(sol_back_con[idx, tsbcon:tfbcon], sol_back_con[idy, tsbcon:tfbcon], sol_back_con[idz, tsbcon:tfbcon], linewidth = 0.5, color = :blue)

#lines!(sol_back[idx, tsb:tfb], sol_back[idy, tsb:tfb], sol_back[idz, tsb:tfb], linewidth = 1.0, color = :black)

lines!(sol_for[idx, tsf:tff], sol_for[idy, tsf:tff], sol_for[idz, tsf:tff], linewidth = 0.5, color = :deeppink)

for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)